In [2]:
"""
Topic Extraction:  ranks each tweet in a pre-defined list of topics.
"""
import pandas as pd
from transformers import pipeline
from tqdm import tqdm

In [2]:
classifier = pipeline('zero-shot-classification', model='facebook/bart-large-mnli')

candidate_labels = ['Healthcare and Pharmaceuticals', 'Technology', 'Energy', 'Financials',
                    'Automotive', 'Agriculture', 'Airlines and Travel',
                    'Cryptocurrency and Fintech',
                    'Defense and Aerospace', 'Arts and Entertainment']

Device set to use mps:0


In [3]:
def classify_tweet(text):
    """
    Scores how likely a tweet refers to a topic.
    """
    # pylint: disable=W0718
    try:
        result = classifier(text, candidate_labels, multi_label=True)

        return dict(zip(result['labels'], result['scores']))
    except Exception as e:
        print(f'Error classifying tweet: {e}')
        return {}
    # pylint: enable=W0718

In [4]:
tweets = pd.read_csv('../data/tweets_with_sentiment.csv')
tweets.head()

,id,user_id,handle,clean_text,lang,timestamp,sentiment_label,sentiment_score
0,1603456812723933194,12.0,jack,"RT @ODELL: @nobsbitcoin love to see it, if yal...",en,2022-12-15 18:27:49.597000+00:00,Neutral,0.977740
1,1600412216687149056,12.0,jack,RT @TheGuySwann: It would be so amazing to see...,en,2022-12-07 08:49:41.336000+00:00,Positive,0.999987
2,1595502702854692866,12.0,jack,RT @kendricklamar: big steppers tour (direct...,en,2022-11-23 19:41:02.024000+00:00,Neutral,0.999613
3,1589450807216599040,12.0,jack,@litcapital @elonmusk Sure,en,2022-11-07 02:52:57.665000+00:00,Neutral,0.985752
4,1582442966521503745,12.0,jack,RT @bluesky: Bluesky is building a social prot...,en,2022-10-18 18:46:18.244000+00:00,Neutral,0.999923


In [ ]:
tqdm.pandas()
tweets['topic_scores'] = tweets['clean_text'].progress_apply(classify_tweet)

 38%|██████████████████████████████                                                  | 1903/5059 [07:43<12:07,  4.34it/s]

In [ ]:
tweets['top_topic'] = tweets['topic_scores'].apply(lambda d: max(d, key=d.get) if d else None)

tweets = pd.concat([tweets, tweets['topic_scores'].apply(pd.Series)], axis=1)
del tweets['topic_scores']

In [ ]:
tweets.head()

In [ ]:
tweets.to_csv('../data/tweets_with_sentiment_and_topic.csv')